In [ ]:
Collab Stuff

In [ ]:
!pip install -U hbp_neuromorphic_platform

In [ ]:
username = ''
password = ''

In [ ]:
import nmpi
client = nmpi.Client(username, password)
print(client.my_collabs().keys())

In [ ]:
aistate_collab_id = client.my_collabs()['UH AI state']['id']

Original

In [ ]:
Gexc = 4.
Ginh = 51.
pconn = 0.02
n = 1500
tstop = 60 * 1000

command = "GridSearch.py {system} %s %s %s %s %s " % (Gexc, Ginh, pconn, n, tstop)
print command

In [ ]:
job_path = client.submit_job(source="https://github.com/sesutton/GridSearch",
                              platform=nmpi.SPINNAKER,
                              collab_id=aistate_collab_id,
                              command=command)

Pooled Version

In [ ]:
def pooled_grid_search(command):  
    job_path = client.submit_job(source="https://github.com/sesutton/GridSearch",
                              platform=nmpi.SPINNAKER,
                              collab_id=aistate_collab_id,
                              command=command) 

In [ ]:
commands = []
for Gexc in range(1,101, 4):
    for Ginh in range(1,101, 4):
        pconn = 0.02
        n = 1500
        tstop = 60 * 1000
        commands.append("GridSearch.py {system} %s %s %s %s %s " % (Gexc, Ginh, pconn, n, tstop))

print len(commands)

In [ ]:
import multiprocessing
p = multiprocessing.Pool()
p.map(pooled_grid_search, commands)

In [ ]:
job_id = 100430 - 625 + 1
ret = client.get_job(job_id, with_log=True)
result = client.job_status(job_id)
print result

ret = client.get_job(job_id, with_log=True)
print ret
#filenames = client.download_data(ret, local_dir=sim_dir)

Serial Version

In [ ]:
def Grid_Search(Gexc, Ginh, pconn, n, tstop):
    command = "GridSearch.py {system} %s %s %s %s %s " % (Gexc, Ginh, pconn, n, tstop)
    print command
    
    job_path = client.submit_job(source="https://github.com/sesutton/GridSearch",
                              platform=nmpi.SPINNAKER,
                              collab_id=aistate_collab_id,
                              command=command)

In [ ]:
for Gexc in range(1,101):
    for Ginh in range(1,101):
            pconn = 0.02
            n = 1500
            tstop = 60 * 1000
            Grid_Search(Gexc, Ginh, pconn, n, tstop)

Download results

In [ ]:
import os

results_dir = os.getcwd() + '/results/'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

job_id = 0
for Gexc in range(1,101):
    for Ginh in range(1,101):
        pconn = 0.02
        n = 1500
        tstop = 6 * 1000
            
        sim_dir = results_dir + "/Gexc=%s Ginh=%s pconn=%s n=%s tstop=%s/" % (Gexc, Ginh, pconn, n, tstop)
        if not os.path.exists(sim_dir):
            os.makedirs(sim_dir)
            
        ret = client.get_job(job_id, with_log=True)
        filenames = client.download_data(ret, local_dir=sim_dir)
        
        job_id += 1

In [ ]:
job_id = job_path.split('/')[-1]
print job_id

In [ ]:
# execute this line until it says 'finished'
import time

result = ''
from IPython.display import clear_output
while True:
    try:
        result = client.job_status(job_id)
    except:
        pass
    clear_output()
    print result
    time.sleep(0.1)
    if result == 'finished' or result == 'error':
        break
    

In [ ]:
ret = client.get_job(job_id, with_log=True)

In [ ]:
print(ret['log'])

In [ ]:
import os

results_dir = os.getcwd() + '/results/'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
sim_dir = results_dir + "/Gexc=%s Ginh=%s pconn=%s n=%s tstop=%s/" % (Gexc, Ginh, pconn, n, tstop)
if not os.path.exists(sim_dir):
    os.makedirs(sim_dir)

In [ ]:
filenames = client.download_data(ret, local_dir=sim_dir)

Remove Jobs from Collab

In [ ]:
def remove_comp_job(job_id):
    try:
        client.remove_completed_job(job_id)
        print "success %s" % (job_id)
    except:
        failed.append(job_id)
        print "fail %s" % (job_id)

def remove_queued_job(job_id):
    try:
        client.remove_queued_job(job_id)
        print "success %s" % (job_id)
    except:
        print "fail %s" % (job_id)
      

In [ ]:
failed = []
first = 98455
last = 98455

import multiprocessing
p = multiprocessing.Pool()
p.map(remove_comp_job, range(first, last + 1))
p.map(remove_queued_job, range(first, last + 1))